In [1]:
from Bio import SeqIO
import os
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import nltk

In [2]:
# 获取FASTA
def getFastaData(fasta_path):
    # 定义FASTA文件路径
    fasta_file = fasta_path

    # 读取FASTA文件
    sequences = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append({
            "id": record.id,
            "sequence": str(record.seq),
        })
    result = []
    # 打印读取到的序列
    for sequence in sequences:
        
        result.append(sequence["sequence"])

    return result

In [3]:
p = getFastaData(r'C:\Users\Admin\Desktop\self trans\data\pos_fasta_lysine.fasta')
n = getFastaData(r'C:\Users\Admin\Desktop\self trans\data\neg_fasta_lysine.fasta')
p_label = [1 for i in range(len(p))]
n_label = [0 for i in range(len(n))]
feature = p + n
label = p_label + n_label

In [4]:
test_p = getFastaData(r'C:\Users\Admin\Desktop\self trans\data\pos_ind_testdata.fasta')
test_n = getFastaData(r'C:\Users\Admin\Desktop\self trans\data\neg_ind_testdata.fasta')
test_p_label = [1 for i in range(len(test_p))]
test_n_label = [0 for i in range(len(test_n))]
test_feature = test_p + test_n
test_label = test_p_label + test_n_label

In [5]:
anji = []
for fe in feature:
    for it in fe:
        if it not in anji and it != ' ':
            anji.append(it)
# amino_acids = dict()
# for i in range(len(anji)):
#     amino_acids[anji[i]] = i
# test_feature_one = []
# for i in range(len(test_feature)):
#     temp = []
#     for j in range(31):
#         temp_2 = [0 for i in range(20)]
#         temp_2[amino_acids[test_feature[i][j]]] = 1
#         temp.extend(temp_2)
#     test_feature_one.append(temp)


In [6]:
# anji = []
# for fe in feature:
#     for it in fe:
#         if it not in anji and it != ' ':
#             anji.append(it)
# amino_acids = dict()
# for i in range(len(anji)):
#     amino_acids[anji[i]] = i
# feature_one = []
# for i in range(len(feature)):
#     temp = []
#     for j in range(31):
#         temp_2 = [0 for i in range(20)]
#         temp_2[amino_acids[feature[i][j]]] = 1
#         temp.extend(temp_2)
#     feature_one.append(temp)

In [7]:
def getFre(fe):
    temp_dict = {}
    for aj in anji:
        temp_dict[aj] = 0
    for j in range(len(fe)):
        if fe[j] == ' ':
            continue
        temp_dict[fe[j]] += 1
    return [temp_dict[k] for k in anji]

In [8]:
# amino_acids = dict()
# for i in range(len(anji)):
#     amino_acids[anji[i]] = i

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, AutoTokenizer,BertModel, AutoModel 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
class myModel(torch.nn.Module):
    def __init__(self,esm2):
        super(myModel,self).__init__()
        self.esm2 = esm2
        self.fc1 = torch.nn.Linear(1024,16)
        self.fc2 = torch.nn.Linear(16,1)
        
        self.fc31 = torch.nn.Linear(1024,256)
        self.fc32 = torch.nn.Linear(256,128)
        
        self.fc41 = torch.nn.Linear(1024,64)
        self.fc42 = torch.nn.Linear(64,128)
        
        
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()  # 隐藏层的激活函数
        self.bn1 = torch.nn.BatchNorm1d(128)  # 添加 Batch Normalization 层
        self.dropout1 = nn.Dropout(p=0.1)
        self.dropout2 = nn.Dropout(p=0.1)
    def forward(self,x):
        outputs_ = self.esm2(**inputs)

        x = outputs_.last_hidden_state[:, 0, :]
        x = self.dropout1(x)
        x1 = self.fc1(x)
        x1 = self.relu(x1)
        
        x2 = self.fc31(x)
        x2 = self.relu(x2)
        x2 = self.fc32(x2)
        x2 = self.relu(x2)
        
        x3 = self.fc41(x)
        x3 = self.relu(x3)
        x3 = self.fc42(x3)
        x3 = self.relu(x3)
        
        x = x1 + x2 + x3
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        x = self.sigmoid(x)
        return x
        
        

In [10]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, recall_score, precision_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression

def comp_result(y_test, y_pred, y_proba):
    # 计算评估指标
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    mcc = matthews_corrcoef(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # 打印结果
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"MCC: {mcc:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Precision: {precision:.4f}")
    
    return accuracy,f1,auc,mcc,recall,precision

In [11]:
from collections import Counter


# 统计K-mer组成
def kmer_composition(seq, k):
    kmers = [seq[i:i+k] for i in range(len(seq)-k+1)]
    kmer_counts = Counter(kmers)
    return kmer_counts



In [12]:
k = 2
kmer_counts_list = []
for fe in feature:
    kmer_counts = kmer_composition(fe, k)
    for it in kmer_counts:
        if it not in kmer_counts_list:
            kmer_counts_list.append(it)

In [13]:
kmer_counts_dict = {}
for i in range(len(kmer_counts_list)):
    kmer_counts_dict[kmer_counts_list[i]] = i

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, AutoTokenizer,BertModel, AutoModel 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F


class Attention(nn.Module):
    def __init__(self, input_dim):
        super(Attention, self).__init__()
        self.W = nn.Linear(input_dim, 1)

    def forward(self, x):
        attn_weights = F.softmax(self.W(x), dim=1)
        output = attn_weights * x
        return output

class myModel(torch.nn.Module):
    def __init__(self,esm2):
        super(myModel,self).__init__()
        self.esm2 = esm2
        self.fc1 = torch.nn.Linear(1280,32)
        self.fc2 = torch.nn.Linear(64,16)
        self.fc3 = torch.nn.Linear(16,1)
    
        self.att1 = Attention(64)
        self.att2 = Attention(16)
        
        
        
        self.sigmoid = nn.Sigmoid()
        self.relu = nn.ReLU()  # 隐藏层的激活函数
        self.bn1 = torch.nn.BatchNorm1d(128)  # 添加 Batch Normalization 层
        self.dropout1 = nn.Dropout(p=0.2)
        self.dropout2 = nn.Dropout(p=0.5)
        
        
        
        self.fc_anji1 = torch.nn.Linear(420,64)
        self.fc_anji2 = torch.nn.Linear(64,32)
        
        
        
        self.fz_fc1 = torch.nn.Linear(32,1)
        self.fz_fc3 = torch.nn.Linear(32,1)
        
    def forward(self,x,batch_input_anji):
        outputs_ = self.esm2(**inputs)

        x = outputs_.last_hidden_state[:, 0, :]
        x = self.dropout2(x)
        x = self.fc1(x)
        x = self.relu(x)
        
        fx_1 = self.fz_fc3(x)
        fx_1 = self.sigmoid(fx_1)
        
        x = self.dropout2(x)
        
        batch_input_anji = self.fc_anji1(batch_input_anji)
        batch_input_anji = self.relu(batch_input_anji)
        batch_input_anji = self.fc_anji2(batch_input_anji)
        batch_input_anji = self.relu(batch_input_anji)
        
        fx_2 = self.fz_fc1(batch_input_anji)
        fx_2 = self.sigmoid(fx_2)
        
        batch_input_anji = self.dropout2(batch_input_anji)
        
        
        

        
        x = torch.cat((batch_input_anji, x), dim=1)

        x = self.relu(x)
        x = self.dropout2(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x,fx_1,fx_2
        
        

In [15]:
# 定义测试集
batch_size = 1  # 每个批次的样本数量
val_data_loader = DataLoader(list(zip(test_feature, test_label)), batch_size=batch_size, shuffle=True)

In [ ]:
from sklearn.model_selection import KFold
import numpy as np
import copy as cp
from sklearn.model_selection import StratifiedKFold
import pandas as pd
# 定义五折交叉验证
torch.manual_seed(42)
np.random.seed(42)
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
kf_in = 0

feature = np.array(feature)
label = np.array(label)
# 迭代生成每一折的训练和测试数据索引
for train_index, test_index in stratified_kfold.split(feature, label):
    
    save_model = None
    best_accuracy = 0
    best_f1 = 0
    best_auc = 0
    best_mcc = 0
    best_recall = 0
    best_precision = 0
    

    best_test_labels_com = None
    best_predict_labels_com = None
    best_test_outputs_com = None
    best_intermediate_outputs = None
    
    
    
    print(train_index)
    kf_in += 1
    print('kf',kf_in)
    train_features, test_features = list(feature[train_index]), list(feature[test_index])
    train_labels, test_labels = list(label[train_index]), list(label[test_index])
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 指定保存位置的路径
    cache_directory = r"C:\Users\Admin\Desktop\neuropeptide prediction\model"

    # 加载 tokenizer 和模型
    tokenizer_ = AutoTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D", cache_dir=cache_directory)
    esm2 = AutoModel.from_pretrained("facebook/esm2_t33_650M_UR50D", cache_dir=cache_directory)
    esm2 = esm2.to(device)
    del esm2.encoder.layer[4:]
    
    model = myModel(esm2)
    model = model.to(device)
    
    # 定义损失函数和优化器
    criterion = nn.BCELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)


    # 将数据包装成TensorDataset，并使用DataLoader进行批次训练
    
    data_loader = DataLoader(list(zip(train_features, train_labels)), batch_size=batch_size, shuffle=True)


    test_data_loader = DataLoader(list(zip(test_features,test_labels)), batch_size=batch_size, shuffle=True)
    
    
    # 训练模型
    num_epochs = 50
    
    for epoch in range(num_epochs):
        total_loss = 0
        for batch_input, batch_labels in data_loader:
        
            batch_input_list = list(batch_input)
            
            batch_input_anji = []
            for i in range(len(batch_input_list)):
                batch_input_anji.append(getFre(batch_input_list[i]))
                
            for i in range(len(batch_input_list)):
                kmer_counts = kmer_composition(batch_input_list[i], k)
                feature_temp = [0 for i in range(400)]
                for it in kmer_counts:
                    feature_temp[kmer_counts_dict[it]] = kmer_counts[it]
                batch_input_anji[i].extend(feature_temp)
    
            batch_input_anji = torch.tensor(batch_input_anji).float()
            batch_input_anji = batch_input_anji.to(device)
            
                
            inputs = tokenizer_(batch_input, return_tensors='pt', padding=True, truncation=True)
            inputs = inputs.to(device)
            outputs = model(inputs,batch_input_anji)
        
            outputs1 = outputs[0].view(-1)
            outputs2 = outputs[1].view(-1)
            outputs3 = outputs[2].view(-1)
            loss1 = criterion(outputs1.to('cpu'), batch_labels.to(torch.float32))
            loss2 = criterion(outputs2.to('cpu'), batch_labels.to(torch.float32))
            loss3 = criterion(outputs3.to('cpu'), batch_labels.to(torch.float32))
            loss = loss1 + loss2*0.3 + loss3*0.3
            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # 在每个训练周期之后进行测试集性能评估
        model.eval()  # 切换模型为评估模式
        test_loss = 0
        test_loss1 = 0
        test_loss2 = 0
        test_loss3 = 0
        test_loss4 = 0
        correct_predictions1= 0
        correct_predictions2= 0
        correct_predictions3= 0
        correct_predictions4= 0
        
        test_labels_com = []
        predict_labels_com = []
        test_outputs_com = []
        intermediate_outputs = []
        
        with torch.no_grad():
            for test_batch_input,test_batch_labels in test_data_loader:
                
                batch_input_list = list(test_batch_input)
            
                batch_input_anji = []
                for i in range(len(batch_input_list)):
                    batch_input_anji.append(getFre(batch_input_list[i]))

                    
                for i in range(len(batch_input_list)):
                    kmer_counts = kmer_composition(batch_input_list[i], k)
                    feature_temp = [0 for i in range(400)]
                    for it in kmer_counts:
                        feature_temp[kmer_counts_dict[it]] = kmer_counts[it]
                    batch_input_anji[i].extend(feature_temp)
                batch_input_anji = torch.tensor(batch_input_anji).float()
                batch_input_anji = batch_input_anji.to(device)
                

                inputs = tokenizer_(test_batch_input, return_tensors='pt', padding=True, truncation=True)
                inputs = inputs.to(device)

                test_outputs = model(inputs,batch_input_anji)
                
        
                test_outputs1 = test_outputs[0].view(-1)
                test_outputs2 = test_outputs[1].view(-1)
                test_outputs3 = test_outputs[2].view(-1)
        
                
        
                loss1 = criterion(test_outputs1.to('cpu'), test_batch_labels.to(torch.float32)).item()
                loss2 = criterion(test_outputs2.to('cpu'), test_batch_labels.to(torch.float32)).item()
                loss3 = criterion(test_outputs3.to('cpu'), test_batch_labels.to(torch.float32)).item()
                test_loss1 += loss1
                test_loss2 += loss2
                test_loss3 += loss3
                
                test_loss += (loss1 + loss2*0.3 + loss3*0.3)
                
                predictions1 = (test_outputs1 > 0.5).float()  # 大于0.5的预测为1，否则为0
                predictions2 = (test_outputs2 > 0.5).float()  # 大于0.5的预测为1，否则为0
                predictions3 = (test_outputs3 > 0.5).float()  # 大于0.5的预测为1，否则为0
                
                correct_predictions1 += (predictions1.to('cpu') == test_batch_labels).sum().item()
                correct_predictions2 += (predictions2.to('cpu') == test_batch_labels).sum().item()
                correct_predictions3 += (predictions3.to('cpu') == test_batch_labels).sum().item()
                
                predict_labels_com.extend(predictions1.tolist())
                test_labels_com.extend(test_batch_labels.tolist())
                test_outputs_com.extend(test_outputs1.tolist())
        
        average_test_loss = test_loss / len(val_data_loader)
        average_test_loss1 = test_loss1 / len(val_data_loader)
        average_test_loss2 = test_loss2 / len(val_data_loader)
        average_test_loss3 = test_loss3 / len(val_data_loader)
        accuracy1 = correct_predictions1 / len(test_feature)
        accuracy2 = correct_predictions2 / len(test_feature)
        accuracy3 = correct_predictions3 / len(test_feature)
        
        
        
        
        predict_labels_com = np.array(predict_labels_com)
        test_labels_com = np.array(test_labels_com)
        test_outputs_com = np.array(test_outputs_com)
        temp_accuracy,temp_f1,temp_auc,temp_mcc,temp_recall,temp_precision = comp_result(test_labels_com,predict_labels_com,test_outputs_com)
        if temp_accuracy > best_accuracy:
            best_accuracy = temp_accuracy
            best_f1 = temp_f1
            best_auc = temp_auc
            best_mcc = temp_mcc
            best_recall = temp_recall
            best_precision = temp_precision

            
            best_test_labels_com = test_labels_com
            best_predict_labels_com = predict_labels_com
            best_test_outputs_com = test_outputs_com
            best_intermediate_outputs = intermediate_outputs
            save_model = cp.deepcopy(model)
            
            temp_output_pd = pd.DataFrame(best_test_outputs_com)
            temp_label_pd = pd.DataFrame(best_test_labels_com)

            temp_output_pd.to_csv(r'C:\Users\Admin\Desktop\self trans\result\new_Fold%d_output.csv' % kf_in)
            temp_label_pd.to_csv(r'C:\Users\Admin\Desktop\self trans\result\new_Fold%d_label.csv' % kf_in)
        
        average_train_loss = total_loss / len(data_loader)
        
        
        # 测试集
        print("测试集================")
        test_labels_com = []
        predict_labels_com = []
        test_outputs_com = []
        intermediate_outputs = []
        
        with torch.no_grad():
            for test_batch_input, test_batch_labels in val_data_loader:
                
                batch_input_list = list(test_batch_input)
            
                batch_input_anji = []
                for i in range(len(batch_input_list)):
                    batch_input_anji.append(getFre(batch_input_list[i]))

                    
                for i in range(len(batch_input_list)):
                    kmer_counts = kmer_composition(batch_input_list[i], k)
                    feature_temp = [0 for i in range(400)]
                    for it in kmer_counts:
                        feature_temp[kmer_counts_dict[it]] = kmer_counts[it]
                    batch_input_anji[i].extend(feature_temp)
                batch_input_anji = torch.tensor(batch_input_anji).float()
                batch_input_anji = batch_input_anji.to(device)
                

                inputs = tokenizer_(test_batch_input, return_tensors='pt', padding=True, truncation=True)
                inputs = inputs.to(device)

                test_outputs = model(inputs,batch_input_anji)
                
        
                test_outputs1 = test_outputs[0].view(-1)
       
                predictions1 = (test_outputs1 > 0.5).float()  # 大于0.5的预测为1，否则为0
                
                
                predict_labels_com.extend(predictions1.tolist())
                test_labels_com.extend(test_batch_labels.tolist())
                test_outputs_com.extend(test_outputs1.tolist())

        predict_labels_com = np.array(predict_labels_com)
        test_labels_com = np.array(test_labels_com)
        test_outputs_com = np.array(test_outputs_com)
        comp_result(test_labels_com,predict_labels_com,test_outputs_com)
        print("======================")
        # 切回训练模式
        model.train()
        print('Epoch [{}/{}], Train Loss: {:.4f}, Test Loss: {:.4f}, Test Loss1: {:.4f}, Test Loss2: {:.4f}, Test Loss3: {:.4f}, Accuracy1: {:.2f}, Accuracy2: {:.2f}, Accuracy3: {:.2f}'.format(
            epoch + 1, num_epochs, average_train_loss, average_test_loss, average_test_loss1, average_test_loss2, average_test_loss3, accuracy1 * 100, accuracy2 * 100, accuracy3 * 100))
    torch.save(save_model, r'C:\Users\Admin\Desktop\self trans\reuslt_model\new_best_model_%d.pth' % kf_in)
    print('=====================================')
    print('kf',kf_in,"best result:")
    print('acc: {:.4f}'.format(best_accuracy))
    print('f1: {:.4f}'.format(best_f1))
    print('auc： {:.4f}'.format(best_auc))
    print('mcc: {:.4f}'.format(best_mcc))
    print('recall：{:.4f}'.format(best_recall))
    print('prec: {:.4f}'.format(best_precision))
    print('=====================================')
    

[    0     2     3 ... 13946 13947 13948]
kf 1


Some weights of EsmModel were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
